# Example: Single Agent

Install SUMO.

In [ ]:
# Setup SUMO=1.10.0
!apt-get update --fix-missing
!apt-get install -y libspatialindex-dev
%pip install --upgrade pip wheel
%pip install eclipse-sumo==1.10.0
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo

Install SMARTS.

In [ ]:
# Install SMARTS
%cd ~
!rm -rf /content/SMARTS
!git clone https://github.com/huawei-noah/SMARTS /content/SMARTS
!cd /content/SMARTS && git checkout 'ipynb-1' && pip install .[camera-obs]
!echo -e "import sys\nsys.path.insert(0, '/content/SMARTS/')" | python

Build the scenarios.

In [ ]:
# Build scenarios
!scl scenario build-all --clean /content/SMARTS/scenarios/figure_eight

Define the agent.

In [ ]:
import gym

from smarts.core.agent import Agent
from smarts.core.sensors import Observation

class ChaseViaPointsAgent(Agent):
    def act(self, obs: Observation):
        if (
            len(obs.via_data.near_via_points) < 1
            or obs.ego_vehicle_state.road_id != obs.via_data.near_via_points[0].road_id
        ):
            return (obs.waypoint_paths[0][0].speed_limit, 0)

        nearest = obs.via_data.near_via_points[0]
        if nearest.lane_index == obs.ego_vehicle_state.lane_index:
            return (nearest.required_speed, 0)

        return (
            nearest.required_speed,
            1 if nearest.lane_index > obs.ego_vehicle_state.lane_index else -1,
        )

Define the environment loop.

In [ ]:
from smarts.core.agent import AgentSpec
from smarts.core.agent_interface import AgentInterface, AgentType
from smarts.core.utils.episodes import episodes
from smarts.env.hiway_env import HiWayEnv
from smarts.env.wrappers.single_agent import SingleAgent

AGENT_ID = "SingleAgent"

def main(scenarios, sim_name, headless, num_episodes, seed, envision_endpoint, max_episode_steps=None):
    agent_spec = AgentSpec(
        interface=AgentInterface.from_type(
            AgentType.LanerWithSpeed, max_episode_steps=max_episode_steps
        ),
        agent_builder=ChaseViaPointsAgent,
    )

    env = HiWayEnv(
        scenarios=scenarios,
        agent_specs={AGENT_ID: agent_spec},
        sim_name=sim_name,
        headless=headless,
        visdom=False,
        fixed_timestep_sec=0.1,
        sumo_headless=True,
        seed=seed,
        envision_endpoint=envision_endpoint
    )

    # Wrap a single-agent env with SingleAgent wrapper to make `step` and `reset`
    # output compliant with gym spaces.
    env = SingleAgent(env)

    for episode in episodes(n=num_episodes):
        agent = agent_spec.build_agent()
        observation = env.reset()
        episode.record_scenario(env.scenario_log)

        done = False
        while not done:
            agent_action = agent.act(observation)
            observation, reward, done, info = env.step(agent_action)
            episode.record_step(observation, reward, done, info)

    env.close()

Run the Envision server in the background.

Create an iframe to display the browser running Envision.

Run the example.

In [ ]:
%cd /content/SMARTS

!nohup scl envision start -s /content/SMARTS/scenarios --port=8081 &

from google.colab import output
port=8081
output.serve_kernel_port_as_window(port)


In [ ]:
%cd /content/SMARTS
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

!pip install pyngrok

from pyngrok import ngrok
# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port = '8081')
public_url

In [ ]:
main(
    scenarios=["/content/SMARTS/scenarios/figure_eight"],
    sim_name="Single agent",
    headless=False,
    num_episodes=3,
    seed=42,
    envision_endpoint="ws://localhost:8081"
)

In [ ]:
# get_ipython().system_raw("scl envision start -s /content/SMARTS/scenarios")

# %%bash --bg
# scl envision start -s /content/SMARTS/scenarios &> /content/SMARTS/envision.log

!nohup scl envision start -s /content/SMARTS/scenarios &

In [ ]:
from google.colab import output
port=8081
output.serve_kernel_port_as_iframe(port)

In [ ]:
%killbgscripts
!pkill -9 -f envision
!pkill -9 -f smarts